In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import random
import mplfinance 
import fx
import fx_rl
from collections import deque
from renkodf import Renko
from scipy.signal import lfilter
from deap import base, creator, tools
from scipy.stats import zscore
from tqdm.notebook import tqdm
from datetime import time

In [7]:
# convert date to datetime
# df_full['datetime'] = pd.to_datetime(df_full['date'] + ' ' + df_full['time'], format='%Y.%m.%d %H:%M')
# filter the data to just 2023

# df_2022 = fx.prep_data(df_full, 2022)
# df_2023 = fx.prep_data(df_full, 2023)
# df_2024 = fx.prep_data(df_full, 2024)

# Parameters

In [8]:
# # Renko variable
# initial_brick_size = 0.0003
# # create a list of possible brick sizes a max at 0.001 and min at 0.00001 and each step is 0.00001
# brick_size_list = np.arange(0.0005, 0.00101, 0.00001)
# brick_size_str = str(int(initial_brick_size*10000))

# # sma variables
# initial_sma_length = 3
# sma_length_list = np.arange(3, 20, 1)
# initial_smoothing_sma = 3
# smoothing_sma_list = np.arange(3, 20, 1)


In [30]:
# Lot Size
lot_size = 1
per_lot = 100_000

# stop loss
stop_loss_size_init = 0.00033*4
# threshold for entry
threshold_init = 0.00033*3

# take profit
take_profit_size_init = 0.00033*5

# Commissions
msolutions_commission = lot_size * -5

# starting balance
balance = 200_000

# testing conditions
timeframe_list = ['5min', '15min', 'hourly']
comparisons = ['5min,15_min,2', '5min,hourly,11', '15min,hourly,3']

# Optimization

In [4]:
def check_5_min_data(time, direction, tp_price, sl_price):
    filtered_15min_data = (pl.scan_csv('EURUSD_full_tickstory_data_5_min.csv')
        .with_columns(
            pl.col('').alias('datetime').str.to_datetime(format='%Y-%m-%d %H:%M:%S'),
        ).drop('')
        .filter(
            (pl.col('datetime') <= time)
        )                      
    ).rename(lambda col_name: col_name.lower()).sort('datetime').tail(12).collect()

    result = filtered_15min_data.select([
        pl.col('datetime'),
        (((pl.col('high') >= tp_price) & (direction == 'buy')) |
         ((pl.col('low') <= tp_price) & (direction == 'sell'))).alias('take_profit_hit'),
        (((pl.col('low') <= sl_price) & (direction == 'buy')) |
         ((pl.col('high') >= sl_price) & (direction == 'sell'))).alias('stop_loss_hit'),
    ]).filter(
        pl.col('take_profit_hit') | pl.col('stop_loss_hit')
    ).sort('datetime').head(1)

    if len(result) == 0:
        # print(time, tp_price, sl_price)
        return [tp_price, True]

    if result.row(0)[1]:
        return [tp_price, False]
    else: #result.row(0)[2]:
        return [sl_price, False]



In [32]:
def find_exit_price(row, full_data) -> pl.List:
    position_type, entry_time, stop_loss_price, take_profit_price = row
    entry_time = pd.to_datetime(entry_time)

    # Filter the full_data to only include rows where 'datetime' is > entry_time
    filtered_df = full_data.filter(pl.col('datetime') > entry_time)

    # Check all exit conditions
    result = filtered_df.select([
        pl.col('datetime'),
        pl.col('high'),
        pl.col('low'),
        pl.col('open'),
        pl.col('close'),
        pl.col('week_transition'),
        (((pl.col('high') >= take_profit_price) & (position_type == 'buy')) |
         ((pl.col('low') <= take_profit_price) & (position_type == 'sell'))).alias('take_profit_hit'),
        (((pl.col('low') <= stop_loss_price) & (position_type == 'buy')) |
         ((pl.col('high') >= stop_loss_price) & (position_type == 'sell'))).alias('stop_loss_hit'),
        (pl.col('week_transition') == 1).alias('week_transition_hit'),
        ((pl.col('datetime').dt.time() == time(0,0,0)) & 
         (pl.lit(position_type) == 'buy')).alias('swap_hit')
    ]).filter(
        pl.col('take_profit_hit') | pl.col('stop_loss_hit') | pl.col('week_transition_hit') | pl.col('swap_hit')
    ).sort('datetime').head(1)
    neither_hit = False
    
    if len(result) == 0:
        return [None, None, neither_hit]

    exit_row = result.row(0)
    # print(exit_row)
    exit_time = exit_row[0]
    if exit_row[-4]: # take_profit_hit
        if exit_row[-3]:
            exit_price, neither_hit = check_5_min_data(exit_row[0], position_type, take_profit_price, stop_loss_price)
        exit_price = take_profit_price
    elif exit_row[-3]: # stop_loss_hit
        exit_price = stop_loss_price
    elif exit_row[-2]: # week_transition_hit
        exit_price = exit_row[4] # close price
    elif exit_row[-1]: # swap_hit
        exit_price = exit_row[3] # open price
    else:
        return [None, None, neither_hit]

    return [exit_time, exit_price, neither_hit]

In [24]:
def objective(comparison_str, threshold, take_profit_size, stop_loss_size):
    comparison = comparison_str.split(',')
    try:
        df_joined = (
            pl.scan_csv(f"df_w_news_{comparison[0]}_pred.csv")
            # convert 'Time' to datetime
            .with_columns([
                pl.col('datetime').str.to_datetime(format='%Y-%m-%d %H:%M:%S'),#T %.f
            ])
        .sort('ds')
            ).collect()
    except:
        df_joined = (
            pl.scan_csv(f"df_w_news_{comparison[0]}_pred.csv")
            # convert 'Time' to datetime
            .with_columns([
                pl.col('datetime').str.to_datetime(format='%Y-%m-%dT%H:%M:%S%.f'),#T %.f
            ])
        .sort('ds')
            ).collect()

    # df_processed = fcst.preprocess(df_joined, static_features=[])
    max_date_1hr = df_joined.select(pl.col('datetime').max()).item()
    testing = False
    if testing:
        train, validation, test = fx_rl.slices_finder_polars(df_joined, max_date_1hr, date_col='datetime', testing_needed=testing)
        train = train.select(*df_joined.columns)#.drop("datetime")
        validation = validation#.drop("datetime")
        test = test#.drop("datetime")
    else:
        train, validation = fx_rl.slices_finder_polars(df_joined, max_date_1hr, date_col='datetime', testing_needed=testing)
        train = train.select(*df_joined.columns)#.drop("datetime")
        validation = validation#.drop("datetime")

    # create a column called Pred_lag1 which is the lag of the Pred column in polars
    df_joined_lag3 = train.with_columns(
        pl.col("pred").shift(-int(comparison[2])).alias("pred_lag"),
    ).drop_nulls(subset=["pred_lag"])

    try: 
        full_ohlc_hourly_df = (
            pl.scan_csv(f'EURUSD_full_tickstory_data_{comparison[1]}.csv')# 15_min
            .with_columns([
                pl.col('Time').alias('datetime').str.to_datetime(format='%Y-%m-%d %H:%M:%S+00:00'),
            ])
            .drop('Time')
        ).rename(lambda col_name: col_name.lower()).collect()
    except:
        full_ohlc_hourly_df = (
            pl.scan_csv(f'EURUSD_full_tickstory_data_{comparison[1]}.csv')
            .with_columns([
                pl.col('').alias('datetime').str.to_datetime(format='%Y-%m-%d %H:%M:%S'),
            ])
            .drop('')
        ).rename(lambda col_name: col_name.lower()).collect()  

    df_joined_hourly = df_joined_lag3.join(full_ohlc_hourly_df, on='datetime', how='inner')   

    # create the buy and sell signals which is that if the Pred_lag1 is greater than the y by threshold then buy and if the 
    # Pred_lag1 is less than the y by threshold then sell
    df_joined_bs = df_joined_hourly.with_columns(
        (pl.col("pred_lag") > pl.col("y") + threshold).cast(pl.Int8).alias("buy"),
        (pl.col("pred_lag") < pl.col("y") - threshold).cast(pl.Int8).alias("sell"),
    )

    # create a column called signal which is the buy column if the buy column is 1 and the sell column is 0 
    # and the sell column * -1 if the buy column is 0 and the sell column is 1 and 0 if the buy column is 0 and the sell column is 0
    df_joined_signal = df_joined_bs.with_columns(
        (pl.col("buy") + (pl.col("sell") * -1)).alias("signal"),
    )
    # adjust signal to have the word 'buy' if it is 1 and 'sell' if it is -1 and "" if it is 0 for df_joined_signal which is a polars dataframe
    df_joined_signal_txt = df_joined_signal.with_columns(
        pl.when(pl.col('signal') == 1)
        .then(pl.lit('buy'))
        .when(pl.col('signal') == -1)
        .then(pl.lit('sell'))
        .otherwise(pl.lit(None))
        .alias('signal_text'),
        # add a column for the day of the week 
        pl.col('datetime').dt.weekday().alias('day_of_week'),
        pl.col('datetime').dt.week().alias('week_of_year'),
    )
    # add a column to indicate a transition between weeks by placing a 1 in the column if the week of year changes in the next row
    df_joined_week_trans = df_joined_signal_txt.with_columns(
        pl.when((pl.col('week_of_year') != pl.col('week_of_year').shift(-1)) |
        (pl.col('week_of_year') != pl.col('week_of_year').shift(1)) |
        (pl.col('week_of_year') != pl.col('week_of_year').shift(2)) |
        (pl.col('week_of_year') != pl.col('week_of_year').shift(3)))
        .then(pl.lit(1))
        .otherwise(pl.lit(0))
        .alias('week_transition')
    )

    # if the week_transition is 1 and signal is not 0 then replace signal_text with 'exit' for the df_joined_week_trans polars dataframe
    df_joined_week_trans_exit = df_joined_week_trans.with_columns(
        pl.when(
            (pl.col("week_transition") == 1) & (pl.col("signal") != 0)
        )
        .then(pl.lit("exit"))
        .otherwise(pl.col("signal_text")).alias("signal_text")
    )

    # filter to those rows where the signal_text is not null, exit
    positions = df_joined_week_trans_exit.filter(
        (pl.col('signal_text').is_not_null()) & 
        (pl.col('signal_text') != 'exit') & 
        ((abs(pl.col('seconds_since_last_news_event')) > 900) &
        (abs(pl.col('seconds_to_next_news_event')) > 900)) &
        ((pl.col('signal_text').shift().is_null()) | (pl.col('signal_text').shift() != pl.col('signal_text')))
        ).with_columns(
        pl.when(pl.col('signal_text') == 'buy')
        .then(pl.col('close') + take_profit_size)
        .when(pl.col('signal_text') == 'sell')
        .then(pl.col('close') - take_profit_size)
        .otherwise(None)
        .alias('take_profit'),
        pl.when(pl.col('signal_text') == 'buy')
        .then(pl.col('close') - stop_loss_size)
        .when(pl.col('signal_text') == 'sell')
        .then(pl.col('close') + stop_loss_size)
        .otherwise(None)
        .alias('stop_loss')
        ).select(
            pl.col('signal_text').alias('direction'),
            pl.col('datetime').alias('entry_time'),
            pl.col('close').alias('entry_price'),
            pl.col('take_profit'),
            pl.col('stop_loss'),
        )

    exit_times = []
    exit_prices = []
    neither_hit_bool = []
    for row in positions.iter_rows():
        exit_time, exit_price, neither_hit = find_exit_price([row[0], row[1], row[4], row[3]], df_joined_week_trans_exit)
        exit_times.append(exit_time)
        exit_prices.append(exit_price)
        neither_hit_bool.append(neither_hit)
        
    positions_with_exit_info = positions.with_columns([
        pl.Series("exit_time", exit_times),
        pl.Series("exit_price", exit_prices),
        pl.Series("neither_hit", neither_hit_bool)
    ])

    # profit calculations
    profit_df = positions_with_exit_info.with_columns(
        pl.when(pl.col('direction') == 'buy')
        .then(
            (((pl.col("exit_price") - pl.lit(max(0., np.random.normal(0.0001, 0.00003)))) - 
            (pl.col("entry_price") + pl.lit(max(0., np.random.normal(0.0001, 0.00003))))) * per_lot * lot_size) + msolutions_commission
        )
        .otherwise(
            (((pl.col("entry_price") - pl.lit(max(0., np.random.normal(0.0001, 0.00003)))) -
            (pl.col("exit_price") + pl.lit(max(0., np.random.normal(0.0001, 0.00003)))))  * per_lot * lot_size) + msolutions_commission
            ).alias("profit")
    )
    profit_df_daily = profit_df.group_by_dynamic('entry_time', every='1d').agg(
        pl.col('profit').sum().alias('profit')
    )
    profit_df_daily_drawdown = profit_df_daily.filter(pl.col('profit') < 0)
    
    total_profit = profit_df_daily['profit'].sum()

    profit_factor = total_profit / abs(profit_df_daily_drawdown['profit'].sum())

    return total_profit, len(profit_df_daily_drawdown), profit_factor


In [21]:
# # lol = [brick_size_list, sma_length_list, smoothing_sma_list] # , lot_sizes_list,  
# def constraint_handler(individual):
#     counter = 0
#     for i, lst in zip(individual, lol):
#         # Ensure parameter is greater than   0
#         if (i <= 0) | ((counter == len(individual) - 1) & (i < 2)):
#             individual[counter] = np.random.choice(lst)  # Reset to a valid value
#         counter += 1
#     return individual

In [ ]:
# Define the fitness function
import sys
# Check if FitnessMax already exists in the __main__ namespace
if 'FitnessMax' not in sys.modules['__main__'].__dict__:
    # If not, create it
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))

if 'Individual' not in sys.modules['__main__'].__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMax)

In [33]:
population_size = 100
global_total_profits = []  # A list to collect total profits from all individuals
global_days_in_drawdowns = []  # A list to collect days in drawdown from all individuals
# Global variables to keep track of the number of evaluations
num_evaluations = 0
discard_threshold = 20  # Discard the first 20 evaluations

toolbox = base.Toolbox()

# Register an attribute generator for each parameter with its own range
toolbox.register("take_profit", np.random.uniform, low=1e-10, high=0.01)
toolbox.register("stop_loss", np.random.uniform, low=1e-10, high=0.01)
toolbox.register("threshold", np.random.uniform, low=1e-10, high=0.01)
toolbox.register("comparison", np.random.choice, comparisons)

# Combine the attribute generators to create an individual
toolbox.register("individual", tools.initCycle, creator.Individual,
                  (toolbox.comparison, toolbox.take_profit, toolbox.stop_loss, toolbox.threshold))# , toolbox.lot_size, 
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# initialize a dataframe for the each evaluation
eval_df = pl.DataFrame(['comparison', 'take_profit', 'stop_loss', 'threshold', 'total_profit', 'days_in_drawdown', 'profit_factor', 'num_evals'])


def eval_func(individual):
    # Assuming these are global variables accessible within the scope of eval_func
    global global_total_profits  # A list to collect total profits from all individuals
    global global_days_in_drawdowns  # A list to collect days in drawdown from all individuals
    global num_evaluations  # A global variable to keep track of the number of evaluations
    global eval_df

    # Unpack individual parameters
    comparison, take_profit, stop_loss, threshold = individual #, lot_size, brick_size
    print(comparison, take_profit, stop_loss, threshold) # , lot_size, brick_size

    # Perform calculations using these parameters
    total_profit, days_in_drawdown, profit_factor = objective(comparison, take_profit, stop_loss, threshold)
    print(total_profit, days_in_drawdown, profit_factor)

    # Update the global lists with the new values
    global_total_profits.append(total_profit)
    # print(global_total_profits)
    global_days_in_drawdowns.append(days_in_drawdown)
    
    num_evaluations +=  1
    # save the results to a dataframe
    try:
        eval_df = pl.concat([eval_df, pl.DataFrame({'comparison': comparison, 'take_profit': take_profit, 'stop_loss': stop_loss, 'threshold': threshold,
                                                    'total_profit': total_profit, 'days_in_drawdown': days_in_drawdown, 'profit_factor': profit_factor,
                                                    'num_evals': num_evaluations} 
                                                    )])
    except:
        eval_df = pl.DataFrame({'comparison': comparison, 'take_profit': take_profit, 'stop_loss': stop_loss, 'threshold': threshold,
                                                    'total_profit': total_profit, 'days_in_drawdown': days_in_drawdown, 'profit_factor': profit_factor,
                                                    'num_evals': num_evaluations}
                                                    )


    if len(global_total_profits) > discard_threshold:
        # Normalize total_profit and days_in_drawdown independently
        norm_total_profit = zscore(global_total_profits)[-1]
        norm_days_in_drawdown = zscore(global_days_in_drawdowns)[-1]
        # Combine the scores
        score = norm_total_profit - norm_days_in_drawdown
        if norm_total_profit == 0:
            score = 0
        print(score)
        return (score,)
    else:
        print(num_evaluations)
        return (0,)

toolbox.register("evaluate", eval_func)

# Define the feasibility function
# def feasible(individual):
#     for i, lst in zip(individual, lol):
#         # find the min of the list
#         min = lst.min()
#         # find the max of the list
#         max = lst.max()
#         if i < min or i > max:
#             return False
#     return True

# # Define a distance function to the feasibility region
# def distance(individual):
#     distances = 0
#     for i, lst in zip(individual, lol):
#         # find the min of the list
#         min = lst.min()
#         # find the max of the list
#         max = lst.max()
#         if i < min:
#             # find out how far away from the min the parameter is and then normalize it
#             dist = abs(i - min) / (max - min)
#         elif i > max:
#             # find out how far away from the max the parameter is and then normalize it
#             dist = abs(i - max) / (max - min)
#         else:
#             dist = 0
#         distances += dist
#     return distances  # Distance from the feasibility region

# # Decorate the evaluation function with a DeltaPenalty decorator
# toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 1.0, distance))
toolbox.register("mate", tools.cxUniform, indpb=0.5)
# toolbox.register("mutate", tools.mutPolynomialBounded, low=0.00001, up=0.00101, eta=20.0, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
# Register the population and other operators as before
# toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)

# Create initial population
population = toolbox.population(n=population_size)
for ind in population:
    ind.fitness.values = (0,)  # Temporary fitness value

# Initialize the logbook to store statistics
logbook = tools.Logbook()

# Register the statistics you want to track
stats = tools.Statistics(lambda ind: ind.fitness.values)
# stats.register("gen", np.mean)
stats.register("average", np.mean)
stats.register("minimum", np.min)
stats.register("maximum", np.max)
stats.register("stdeviation", np.std)
logbook.header = "gen", "avg", "min", "max", "std"

# Define the convergence criterion
convergence_threshold = 0.01 # Example threshold for fitness improvement
max_generations_without_improvement = 5 # Example maximum generations without improvement

# Initialize variables to track convergence
best_fitness_value = None
generations_without_improvement = 0

# Run the genetic algorithm
NGEN =  50  # Number of generations
CXPB =  0.7  # Crossover probability
MUTPB =  0.2  # Mutation probability

for gen in tqdm(range(NGEN)):
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit


    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if ind.fitness.values == (0,)]
    # print(invalid_ind)
    invalid_ind.extend([ind for ind in offspring if not ind.fitness.valid])
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        # print(type(ind.fitness.values[0]), ind.fitness.values, type(fit), fit)
        ind.fitness.values = fit
    # Update the statistics
    record = stats.compile(offspring)
    # print(record, offspring[0].fitness.values)
    logbook.record(gen=gen, **record)

    # Check for convergence
    current_best_fitness = record["average"] 
    if (best_fitness_value is None) or (abs(current_best_fitness - best_fitness_value) / best_fitness_value < convergence_threshold):
        best_fitness_value = current_best_fitness
        generations_without_improvement = 0
    else:
        generations_without_improvement += 1

    # Termination condition
    if generations_without_improvement >= max_generations_without_improvement:
        print(f"Converged after {gen} generations")
        break

    # Replace population with the offspring
    population[:] = offspring

# Extract the best individual
best_ind = tools.selBest(population,  1)[0]
best_score = best_ind.fitness.values[0]

# save the logbook to a dataframe
logbook_df = pl.DataFrame(logbook)


  0%|          | 0/50 [00:00<?, ?it/s]

5min,hourly,11 0.005945469692266635 0.006899222979130406 0.00926936102282228
-5513.998141447595 35 -0.3443706817929099
1
5min,hourly,11 0.0037638279533541274 0.00550550299756155 0.0075293435421130575
-13454.377796573777 101 -0.2681499277387649
2
15min,hourly,3 0.009492806430667041 0.0026271570674927074 0.0010287743828694313
588.2191394783943 1 4.45551940923866
3
5min,15_min,2 0.0025731725317836313 0.00540888947315219 0.004999449611053709
1127.3602769809047 41 0.05672629122501704
4
15min,hourly,3 0.008979891579685002 0.007479269614572749 0.0020544144387871226
980.6310254859986 2 2.1177516460262176
5
5min,15_min,2 6.290364912541839e-05 0.0007194979081264168 0.002137842333616701


In [34]:
# save logbook_df to a csv file
logbook_df.to_csv('logbook_df.csv')
# save eval_df to a csv file
eval_df.to_csv('eval_df.csv')

In [35]:
print(best_ind, best_score)

[0.0009400000000000012, 4.93980307589126, 4] 0.9054065712093887


In [36]:
best_comparison, best_take_profit, best_stop_loss, best_threshold = best_ind #, best_lot_size,  

In [ ]:
# results_2023 = making_calculations(df_2023, initial_brick_size, best_inc, best_max, best_ma, best_signal, initial_brick_size*2, initial_lot_size)
# results_2024 = making_calculations(df_2024, initial_brick_size, best_inc, best_max, best_ma, best_signal, initial_brick_size*2, initial_lot_size)

In [41]:
# best_parameters = pd.read_excel('C:/Users/WilliamFetzner/Documents/Trading/best_parameters_dsma.xlsx')
# bp_colnames = best_parameters.columns
# results_3mo = making_calculations(best_brick, int(best_sma_length), int(best_smoothing_sma_length), initial_lot_size)
# new_row_added = pd.concat([best_parameters, pd.DataFrame({bp_colnames[0]: [best_brick], bp_colnames[1]: [best_sma_length], bp_colnames[2]: [best_smoothing_sma_length], 
#            bp_colnames[3]: results_3mo[0], bp_colnames[4]: results_3mo[1], bp_colnames[5]: results_3mo[2]})], ignore_index=True)
# # save the best_parameters dataframe to the best_parameters_IP.xlsx file
# new_row_added.to_excel('C:/Users/WilliamFetzner/Documents/Trading/best_parameters_dsma.xlsx', index=False)
# new_row_added

,best_brick,best_sma_length,best_smoothing_sma_length,profit,days_in_drawdown,profit_factor
0,0.00094,4.939803,4,3.000000,117353.167227,119.000000
1,0.00094,4.939803,4,117353.167227,119.000000,0.707879
